In [6]:
import pandas as pd

# Step 1: Create a sample DataFrame to simulate 'player_activity.csv' with required columns
data = {
    "player_id": [101, 102, 103, 104, 105],
    "date": ["2024-10-02", "2024-10-02", "2024-10-16", "2024-10-18", "2024-10-26"],
    "time_slot": ["S1", "S2", "S1", "S1", "S2"],
    "deposit": [1000, 1500, 1200, 300, 800],
    "withdrawal": [200, 500, 400, 100, 300],
    "games_played": [10, 5, 20, 8, 18]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Step 2: Calculate loyalty points based on the provided formula
df['loyalty_points'] = (
    (0.01 * df['deposit']) + 
    (0.005 * df['withdrawal']) + 
    (0.001 * (df['deposit'] - df['withdrawal']).clip(lower=0)) + 
    (0.2 * df['games_played'])
)

# Step 3: Save the DataFrame as 'player_activity.csv' including the new 'loyalty_points' column
df.to_csv('playerdata.csv', index=False)

print("CSV file 'player_activity.csv' created successfully with loyalty points.")


CSV file 'player_activity.csv' created successfully with loyalty points.


In [10]:
df = pd.read_csv('playerdata.csv')

In [11]:
df

,player_id,date,time_slot,deposit,withdrawal,games_played,loyalty_points
0,101,2024-10-02,S1,1000,200,10,13.8
1,102,2024-10-02,S2,1500,500,5,19.5
2,103,2024-10-16,S1,1200,400,20,18.8
3,104,2024-10-18,S1,300,100,8,5.3
4,105,2024-10-26,S2,800,300,18,13.6


In [13]:
import pandas as pd

# Load the data
data = pd.read_csv('playerdata.csv')  # Assuming CSV format with columns: player_id, date, time_slot, deposit, withdrawal, games_played

# Function to calculate loyalty points for a single record
def calculate_loyalty_points(deposit, withdrawal, num_deposits, num_withdrawals, games_played):
    return (0.01 * deposit) + (0.005 * withdrawal) + (0.001 * max((num_deposits - num_withdrawals), 0)) + (0.2 * games_played)

# Filter data by specific date and slot
def filter_data_by_slot(data, date, slot):
    return data[(data['date'] == date) & (data['time_slot'] == slot)]

# Part A Question 1 - Calculate loyalty points for specific slots
specific_slots = [
    ("2023-10-02", "S1"),
    ("2023-10-16", "S2"),
    ("2023-10-18", "S1"),
    ("2023-10-26", "S2")
]
slot_results = {}

for date, slot in specific_slots:
    filtered_data = filter_data_by_slot(data, date, slot)
    filtered_data['loyalty_points'] = filtered_data.apply(
        lambda row: calculate_loyalty_points(row['deposit'], row['withdrawal'], row['num_deposits'], row['num_withdrawals'], row['games_played']), axis=1
    )
    slot_results[(date, slot)] = filtered_data.groupby('player_id')['loyalty_points'].sum()

# Part A Question 2 - Calculate total loyalty points for October and rank players
october_data = data[data['date'].str.startswith("2023-10")]
october_data['loyalty_points'] = october_data.apply(
    lambda row: calculate_loyalty_points(row['deposit'], row['withdrawal'], row['num_deposits'], row['num_withdrawals'], row['games_played']), axis=1
)
player_points = october_data.groupby('player_id').agg(
    total_points=('loyalty_points', 'sum'),
    total_games=('games_played', 'sum')
)
# Sort by points, then by games played if there's a tie
player_points = player_points.sort_values(by=['total_points', 'total_games'], ascending=[False, False])

# Part A Questions 3, 4, 5 - Additional statistics
avg_deposit = october_data['deposit'].mean()
avg_deposit_per_user = october_data.groupby('player_id')['deposit'].mean().mean()
avg_games_per_user = october_data.groupby('player_id')['games_played'].mean().mean()

print("Slot Results:", slot_results)
print("Player Rankings:", player_points)
print("Average Deposit:", avg_deposit)
print("Average Deposit per User:", avg_deposit_per_user)
print("Average Games per User:", avg_games_per_user)
